# DrJava

## TABLE 4.1  Sebaran tingkat kebahayaan
pie chart nya di simpan di google spreadsheet <a href="#">DrJava - Data Eksperimen</a>

### filter lanza
flow chart perlu menambahkan alir untuk filter kebahayaan berdasarkan kriteria lanza dan marinescu

### kapan refactoring berhenti ?
harm and problem class is exists ?
ya : kembali refactoring
no : stop refactoring

### olah data sqlite hasil JCodeOdor
Jangan lupa buat view di db sqlite nya, seperti:
<ol>
    <li>detect_dcoh</li>
    
</ol>

<pre>CREATE VIEW detect_dcoh as
select 
meth.id as method_id,
type.name as type,meth.name,
value
from 
(select ms.id , ms.parent,ms.name,value 
	from Measurables as ms left join Measures as me 
on ms.id = me.measurable
where key ='DispersedCouplingHarmfulness' and value > 0
order by value desc) as meth
left join 
(select id ,name from Measurables  where type ='type' ) as type
on meth.parent = type.id
</pre>



In [11]:
import pandas as pd    
import numpy as np
import sqlite3 as sql

class Harmfulness:
    def __init__(self, db):
        self.conn = sql.connect('sqlite-jcodeodor/database/'+db)

    def dist(self): 
        return pd.read_sql('select name,type,value from detect_dcoh',self.conn)
#     def get_table_warning()     
    def print_excel(self): 
        dist = self.dist()
        return dist
    
    def get_table(self):    
        harm_sql = 'value = 10 '
        problem_sql = 'value >= 7.75 and value < 10'
        trouble_sql = 'value >= 5.5 and value < 7.75'
        warn_sql = 'value >= 3.25 and value < 5.5'
        bother_sql = 'value >= 1 and value < 3.25'
        harm = self.get_total(harm_sql)
        warn = self.get_total(warn_sql)
        trouble = self.get_total(trouble_sql)
        problem = self.get_total(problem_sql)
        bother = self.get_total(bother_sql)
        arr = np.array([ ['Harm', harm], ['Problem', problem], 
        ['Trouble', trouble], ['Warning', warn],
                        ['Bother', bother]])
        df2 = pd.DataFrame(arr, columns=['level', 'frekuensi'])
        return df2

    def get_total(self,level_sql):
        base_sql = 'select count(*) as total from detect_dcoh where'     	
        return pd.read_sql(base_sql +' '+ level_sql,self.conn).total[0]
    
    def is_view_has_created(self):
#         jika view belum dibuat, langsung error bahwa table detect_dcoh belum dibuat
        if(self.get_total('value >= 7.75 and value < 10')):
            print('yes, detect_dcoh has created')            
    
    #spesial case untuk      
    def filter_for_test_function(self):
        pass
    
tahap0 = Harmfulness('1-tahap-0-drjava.sqlite')
tahap1 = Harmfulness('2-tahap-1-drjava.sqlite')

tahap0.is_view_has_created()
tahap1.is_view_has_created()

yes, detect_dcoh has created
yes, detect_dcoh has created


In [13]:
# kriteria smell dco menurut lanza hardly avoidable
# berdasarkan kriteria lanza, beberapa kelas yang terdeteksi berbahaya tidak ikut direfactoring.
# berikut daftar kelas nya.
# tahap1.dist().to_excel("drjava-class-harmfulnes.xlsx")
# tahap1.dist().to_excel("drjava-class-harmfulnes-tahap1-NEW.xlsx")
# diberi NEW, bisi ketimpa dengan yang baru sy edit

In [8]:
class CustomQuery(Harmfulness):    
    def get_number_of_classes(self):                
        measurable_key = 'NOCS'        
        sql_where_method = "WHERE key ='"+measurable_key+"' AND value > 0 ORDER BY value DESC"
        sql_method = "( SELECT ms.id , ms.parent, ms.name, key,value FROM (SELECT * FROM Measurables WHERE name LIKE 'edu.rice.cs%') as ms LEFT JOIN Measures as me on ms.id = me.measurable "+sql_where_method+" )"                
        main_sql = "SELECT meth.id as method_id,meth.name, meth.key, value from "+sql_method+" as meth left join "                
        sql_type = "(SELECT id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                
        sql = main_sql+sql_type
        return pd.read_sql(sql,self.conn)
    
    def list_by_cint(self):                
        measurable_key = 'CINT'
        sql_meth = "(select ms.id , ms.parent,ms.name,key,value from Measurables as ms LEFT JOIN Measures as me on ms.id = me.measurable where key ='"+measurable_key+"' order by value desc)"        
        sql1 = "select meth.id as method_id, type.name as type,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                
#         where_class = "where type.name in ('SelectionManager') "
#         sql = sql1+sql2+where_class
        sql = sql1+sql2
        return pd.read_sql(sql,self.conn)

    def list_by_fanout(self):
        measurable_key = 'FANOUT'
        sql_meth = "(select ms.id , ms.parent,ms.name,key,value from Measurables as ms LEFT JOIN Measures as me on ms.id = me.measurable where key ='"+measurable_key+"' order by value desc)"        
        sql1 = "select meth.id as method_id, type.name as type,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                        
        sql = sql1+sql2
        return pd.read_sql(sql,self.conn)
    
# cint0 = CustomQuery('1-tahap-0-argouml.sqlite')
# cint0.list_by_cint().to_excel("argouml-CINT-setelah-modif.xlsx")
# cquery0 = CustomQuery('1-tahap-0-drjava.sqlite')
# cquery0.get_number_of_classes()
# cquery0.get_number_of_classes().to_excel("argouml-NOCS.xlsx")
# cquery0.get_number_of_classes().to_csv("drjava-NOCS.csv")
cquery1 = CustomQuery('2-tahap-1-drjava.sqlite')


In [4]:
print("branch tahap-0 i.e sesudah refactoring dilakukan ")
tahap0.get_table()

branch tahap-0 i.e sesudah refactoring dilakukan 


,level,frekuensi
0,Harm,22
1,Problem,26
2,Trouble,37
3,Warning,27
4,Bother,0


In [5]:
print("branch tahap-1 i.e sesudah refactoring dilakukan ")
tahap1.get_table()

branch tahap-1 i.e sesudah refactoring dilakukan 


,level,frekuensi
0,Harm,18
1,Problem,21
2,Trouble,39
3,Warning,27
4,Bother,0


In [6]:
tahap1.dist()
# tahap1.dist().to_excel('warning.xlsx')

,name,type,value
0,"setup(java.io.File, java.io.File, java.lang.St...",DrJavaPropertySetup,10.00
1,"run(java.util.ArrayList, java.util.List, edu.r...",edu.rice.cs.drjava.ui.MainFrame$149,10.00
2,testDocLocationAfterSwitch(edu.rice.cs.drjava....,MainFrameTest,10.00
3,"xtestDancingUIFileOpened(java.lang.String, edu...",MainFrameTest,10.00
4,"javadocEnded(java.lang.String, boolean, java.i...",MainFrame.ModelListener,10.00
...,...,...,...
100,"testModifiedByOtherFalse(java.io.File, edu.ric...",GlobalModelIOTest,3.25
101,testCompileAllDifferentSourceRoots(java.io.Fil...,GlobalModelCompileSuccessTest,3.25
102,testCompileClassPathOKDifferentPackages(java.i...,GlobalModelCompileSuccessTest,3.25
103,"closeFileWithoutPrompt(boolean, edu.rice.cs.dr...",AbstractGlobalModel,3.25
